In [2]:
import sys
# adding llms to the system path, make sure to configure the file azureopenai in llms folder with end point details
sys.path.append('C:\\proddev\\research\\aiagentssamples\\llms\\')
import azureopenai
from autogen import GroupChat, GroupChatManager
from autogen import AssistantAgent
from autogen import UserProxyAgent



gpt4_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": azureopenai.config_list,
    "timeout": 120,
}
user_proxy = UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
    code_execution_config=False,
)
engineer = AssistantAgent(
    name="Engineer",
    llm_config=gpt4_config,
    system_message="""Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
""",
)
scientist = AssistantAgent(
    name="Scientist",
    llm_config=gpt4_config,
    system_message="""Scientist. You follow an approved plan. You are able to categorize papers after seeing their abstracts printed. You don't write code.""",
)
planner = AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and a scientist who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist.
""",
    llm_config=gpt4_config,
)
executor = UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "paper",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)
critic = AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.",
    llm_config=gpt4_config,
)
groupchat = GroupChat(
    agents=[user_proxy, engineer, scientist, planner, executor, critic], messages=[], max_round=50
)
manager = GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

user_proxy.initiate_chat(
    manager,
    message="""
find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.
""",
)

Admin (to chat_manager):


find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.


--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

### Plan

1. **Engineer**: Write a script to scrape arXiv for papers related to Large Language Models (LLMs) published in the last week.
2. **Engineer**: Filter the papers to identify their application domains.
3. **Engineer**: Extract relevant information (title, authors, abstract, domain) from the filtered papers.
4. **Scientist**: Review the extracted information to ensure the correct classification of domains.
5. **Engineer**: Create a markdown table with the reviewed information.
6. **Scientist**: Validate the final markdown table for accuracy and completeness.

### Detailed Steps

1. **Engineer**: Write a Python script using the arXiv API to fetch papers related to LLMs published in the last week.
    - Use search t

ChatResult(chat_id=None, chat_history=[{'content': '\nfind papers on LLM applications from arxiv in the last week, create a markdown table of different domains.\n', 'role': 'assistant', 'name': 'Admin'}, {'content': '### Plan\n\n1. **Engineer**: Write a script to scrape arXiv for papers related to Large Language Models (LLMs) published in the last week.\n2. **Engineer**: Filter the papers to identify their application domains.\n3. **Engineer**: Extract relevant information (title, authors, abstract, domain) from the filtered papers.\n4. **Scientist**: Review the extracted information to ensure the correct classification of domains.\n5. **Engineer**: Create a markdown table with the reviewed information.\n6. **Scientist**: Validate the final markdown table for accuracy and completeness.\n\n### Detailed Steps\n\n1. **Engineer**: Write a Python script using the arXiv API to fetch papers related to LLMs published in the last week.\n    - Use search terms like "Large Language Model", "LLM",